---
## 🎯 Practice Exercises

### Exercise 1: Enhanced ReAct with Reasoning Traces

**Task:** Modify the ReAct agent to explicitly show its reasoning.

**Requirements:**
1. Add a tool called `think` that logs reasoning
2. Agent must call `think` BEFORE any action tool
3. Store reasoning traces in custom state
4. Display full reasoning chain to user

**Example output:**
```
Thought: I need to calculate 2+2
Action: calculator("2+2")
Observation: 4
Thought: Now I have the answer
Final Answer: The result is 4
```

**Deliverables:**
- Enhanced ReAct agent with thinking
- Test with 3 multi-step queries
- Compare verbosity vs standard ReAct

---

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated, Literal, Sequence
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage, ToolMessage, SystemMessage
import operator
import os

In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [4]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [5]:
@tool
def calculator(expression: str) -> str:
    """
    Calculate mathematical expressions.
    
    Args:
        expression: Math expression like "2 + 2" or "15 * 37"
    """
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search(query: str) -> str:
    """
    Search for information (simulated).
    
    Args:
        query: The search query
    """
    # Simulated search results
    knowledge = {
        "python": "Python is a high-level programming language created in 1991.",
        "langgraph": "LangGraph is a framework for building stateful multi-actor applications.",
        "react": "ReAct is an agent pattern that combines reasoning and acting."
    }
    
    for key, value in knowledge.items():
        if key in query.lower():
            return value
    
    return "No information found."


@tool
def think(reasoning: str) -> str:
    """Use this tool to explicitly record your step-by-step thinking.
    You MUST call this BEFORE using any other tool or giving final answer."""
    return f"Thought recorded: {reasoning}"

tools = [calculator, search, think]
print("✅ Tools created")

✅ Tools created


In [41]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# Custom State
class ReActTraceState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    reasoning_trace: Annotated[list[str], operator.add]

MessagesState = ReActTraceState

# System prompt
react_prompt = SystemMessage(content="""You are a helpful assistant with step-by-step assistant.

Use calculator for math and search for information.
Think step-by-step before using tools.

You MUST follow this exact pattern for EVERY response:
1. First call the 'think' tool to record your current reasoning
2. Then (and only then) call other tools if needed or give the final answer

Always be very explicit and detailed in your thinking."""
                             )

# Define nodes
def react_assistant(state: MessagesState) -> dict:
    """Agent node - reasons and decides which tool to use."""
    messages = [react_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Route to tools or end."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

# Build graph
react_builder = StateGraph(MessagesState)
react_builder.add_node("assistant", react_assistant)
react_builder.add_node("tools", ToolNode(tools))

react_builder.add_edge(START, "assistant")
react_builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
react_builder.add_edge("tools", "assistant")  # Loop back for multi-step reasoning

react_agent_manual = react_builder.compile(checkpointer=MemorySaver())

print("✅ ReAct agent (manual) created")

✅ ReAct agent (manual) created


In [42]:
# Test function
def test_agent(agent, query: str, agent_name: str = "Agent"):
    sep = "═" * 80
    mid = "─" * 70

    print(f"\n{sep}")
    print(f"QUERY: {query}")
    print(f"{sep}\n")

    result = agent.invoke(
        {"messages": [HumanMessage(content=query)]},
        config={"configurable": {"thread_id": f"test_{agent_name}"}}
    )

    reasoning_trace = []

    print("🧠 Explicit thought:", end=" ")

    # First pass: extract first think call (preview)
    for msg in result["messages"]:
        if isinstance(msg, ToolMessage) and msg.name == "think":
            print(msg.content.replace("Thought recorded:", "").strip())
            break

    print("\nFINAL RESULT:\n")

    for msg in result["messages"]:

        # User
        if isinstance(msg, HumanMessage):
            print(f"👤 User: {msg.content}")
            print(mid)

        # Assistant
        elif isinstance(msg, AIMessage):
            print("🤖 Assistant:")
            if msg.tool_calls:
                print(f"   Tool calls: {[tc['name'] for tc in msg.tool_calls]}")
            elif msg.content.strip():
                print(f"   {msg.content.strip()}")
            print(mid)

        # Tool output
        elif isinstance(msg, ToolMessage):
            print(f"🛠 {msg.name} → {msg.content}")
            print(mid)

            if msg.name == "think":
                reasoning_trace.append(
                    msg.content.replace("Thought recorded:", "").strip()
                )

    print("\nReasoning trace collected:")
    print("...")

    for r in reasoning_trace:
        print("•", r)

    print(f"\n{sep}")
    print("\n(Output is truncated. View as a scrollable)")

    return result["messages"][-1].content


In [43]:
# Multi-step test
test_agent(
    react_agent_manual,
    "I need to calculate 12 + 30 ?",
    "Manual ReAct_76"
)


════════════════════════════════════════════════════════════════════════════════
QUERY: I need to calculate 12 + 30 ?
════════════════════════════════════════════════════════════════════════════════

🧠 Explicit thought: To calculate 12 + 30, I need to add the two numbers together. The first number is 12 and the second number is 30. I will use a calculator to perform this addition.

FINAL RESULT:

👤 User: I need to calculate 12 + 30 ?
──────────────────────────────────────────────────────────────────────
🤖 Assistant:
   Tool calls: ['think']
──────────────────────────────────────────────────────────────────────
🛠 think → Thought recorded: To calculate 12 + 30, I need to add the two numbers together. The first number is 12 and the second number is 30. I will use a calculator to perform this addition.
──────────────────────────────────────────────────────────────────────
🤖 Assistant:
   Tool calls: ['calculator']
──────────────────────────────────────────────────────────────────────
🛠 ca

'The result of the calculation 12 + 30 is 42.'

In [44]:
# Multi-step test
test_agent(
    react_agent_manual,
    "I need to calculate 12 * 9 ?",
    "Manual ReAct_77"
)


════════════════════════════════════════════════════════════════════════════════
QUERY: I need to calculate 12 * 9 ?
════════════════════════════════════════════════════════════════════════════════

🧠 Explicit thought: To calculate 12 * 9, I will use a calculator to find the product of these two numbers.

FINAL RESULT:

👤 User: I need to calculate 12 * 9 ?
──────────────────────────────────────────────────────────────────────
🤖 Assistant:
   Tool calls: ['think']
──────────────────────────────────────────────────────────────────────
🛠 think → Thought recorded: To calculate 12 * 9, I will use a calculator to find the product of these two numbers.
──────────────────────────────────────────────────────────────────────
🤖 Assistant:
   Tool calls: ['calculator']
──────────────────────────────────────────────────────────────────────
🛠 calculator → 108
──────────────────────────────────────────────────────────────────────
🤖 Assistant:
   The result of 12 * 9 is 108.
─────────────────────────

'The result of 12 * 9 is 108.'

In [ ]:
# Multi-step test
test_agent(
    react_agent_manual,
    "I need to calculate 12 / 0 ?",
    "Manual ReAct_78"
)


════════════════════════════════════════════════════════════════════════════════
QUERY: I need to calculate 12 / 0 ?
════════════════════════════════════════════════════════════════════════════════

🧠 Explicit thought: Dividing by zero is undefined in mathematics. Therefore, calculating 12 / 0 will not yield a valid result.

FINAL RESULT:

👤 User: I need to calculate 12 / 0 ?
──────────────────────────────────────────────────────────────────────
🤖 Assistant:
   Tool calls: ['think']
──────────────────────────────────────────────────────────────────────
🛠 think → Thought recorded: Dividing by zero is undefined in mathematics. Therefore, calculating 12 / 0 will not yield a valid result.
──────────────────────────────────────────────────────────────────────
🤖 Assistant:
   Dividing by zero is undefined in mathematics. Therefore, calculating \( 12 / 0 \) will not yield a valid result.
──────────────────────────────────────────────────────────────────────

Reasoning trace collected:
...
• 

'Dividing by zero is undefined in mathematics. Therefore, calculating \\( 12 / 0 \\) will not yield a valid result.'